In [58]:
import pandas as pd
import numpy as np
import json
from pyvi import ViTokenizer, ViPosTagger 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

data = pd.read_json('test copy.json', convert_dates = True)

In [39]:
data

,body,breadcrumbs,source,tag,link,sabo,time,heading
0,"[Ma túy trên ôtô. Ảnh: Hải quan TP HCM., Vào n...","[Pháp luật, An ninh - Trật tự]",Tiền Phong,"[Liu Ming Yang, Jhu Minh Jyun, Ketamine, Máy é...",/vi-sao-lo-500kg-ma-tuy-tu-quang-ninh-lot-vao-...,\n Các đối tượng trong đường dây ma...,2019-05-12T19:11:50.000+07:00,Vì sao lô 500kg ma túy từ Quảng Ninh lọt vào t...
1,"[Cách đây hơn một năm, tôi từng kêu gọi Mark Z...",[Thế giới],Zing,"[Mark Zuckerberg, Xả súng, Phân biệt chủng tộc...",/la-that-bai-lon-nhat-cua-facebook-mark-zucker...,\n Facebook cần một sự điều hành có...,2019-05-10T06:00:00.000+07:00,"Là thất bại lớn nhất của Facebook, Mark Zucker..."
2,[Niềm tin vào giáo dục của người dân đang bị b...,[Giáo dục],Zing,"[Điểm thi, Gian lận, Thí sinh, Thi cử, Kỳ thi,...",/gian-lan-thi-cu-va-ao-vong-con-vua-thi-lai-la...,\n Người lớn có trách nhiệm gì và c...,2019-05-10T06:06:00.000+07:00,Gian lận thi cử và ảo vọng 'con vua thì lại là...
3,"[Dinh dưỡng tinh thần cực kỳ quan trọng, Có mặ...",[Giáo dục],VietnamNet,"[Bùi Trân Phượng, Nguyễn Quốc Vương, Cảm quan,...",/con-sinh-ra-tu-do-va-con-duoc-lam-bat-cu-dieu...,\n 'Tôi vẫn luôn nhớ tôi đã nói với...,2019-05-10T07:03:00.000+07:00,'Con sinh ra tự do và con được làm bất cứ điều...
4,"[1., Khu vườn rộng 180 ha, là tài sản thuộc ""s...","[Văn hóa, Du lịch]",NLĐ,"[Vincent van Gogh, Lữ quán, Eguisheim, Quận 14...",/hoc-nguoi-ta-cach-giu-lai-dau-xua/r/30655373.epi,\n Cái gì đã vào tranh của Van Gogh...,2019-05-10T05:00:00.000+07:00,Học người ta cách giữ lại dấu xưa
...,...,...,...,...,...,...,...,...
26302,"[Ngày 10/5, Ủy ban Kiểm tra Tỉnh ủy Quảng Ngãi...",[Pháp luật],ĐS&PL,"[Thi hành kỷ luật, Khiển trách, Tỉnh ủy, Kỳ họ...",/ky-luat-truong-cong-an-huyen-vi-de-cap-duoi-s...,\n Trưởng công an huyện Bình Sơn bị...,2019-05-10T22:15:00.000+07:00,Kỷ luật Trưởng Công an huyện vì để cấp dưới sa...
26303,"[ Quán karaoke nơi xảy ra vụ việc , Theo đó, k...","[Pháp luật, Hình sự - Dân sự]",BVPL,"[Nguyễn Tư Duy, Vũ Đình Bình, Quán karaoke Tha...",/doi-tuong-dam-tu-vong-nhan-vien-le-tan-trong-...,\n Công an huyện Giao Thủy (Nam Địn...,2019-05-10T22:33:00.000+07:00,Đối tượng đâm tử vong nhân viên lễ tân trong q...
26304,"[Tối 10/5, các lực lượng chức năng đang khẩn t...","[Pháp luật, Hình sự - Dân sự]",VTC,"[Biển Liên Chiểu, Phường Hòa Hiệp Bắc, Tắm biể...",/tam-bien-o-da-nang-nam-sinh-vien-bi-song-cuon...,\n Cùng 2 người bạn ra bãi biển Liê...,2019-05-10T22:34:00.000+07:00,"Tắm biển ở Đà Nẵng, nam sinh viên bị sóng cuốn..."
26305,"[Mới đây, siêu mẫu Hạ Vy chia sẻ hình ảnh hai ...","[Pháp luật, Hình sự - Dân sự]",VietnamNet,"[Hạ Vy, Ford 7, Lạc Trung, Thanh nhàn, Xây xát...",/con-trai-sieu-mau-ha-vy-bi-o-to-tong-chan-don...,"\n Siêu mẫu Hạ Vy cho biết, hiện co...",2019-05-10T22:24:00.000+07:00,Con trai siêu mẫu Hạ Vy bị ô tô tông chấn động...


In [40]:
#only use 1 trait for each breadcrumbs
breadcrumbs_simplify = []
for i in data.breadcrumbs:
    breadcrumbs_simplify.append(i[0])
s = pd.Series(breadcrumbs_simplify)
#include all of the different breadcrumbs
list_of_breadcrumbs = s.value_counts()
list_of_breadcrumbs.index[:7]

Index(['Giải trí', 'Đời sống', 'Công nghệ ', 'Nhà đất', 'Pháp luật', 'Xe cộ',
       'Khoa học'],
      dtype='object')

In [41]:
#put body into a list
body_preprocessing = []
for i in data.body:
    body_preprocessing.append(i)
body_preprocessing[:5]

[['Ma túy trên ôtô. Ảnh: Hải quan TP HCM.',
  'Vào ngày 11/5, cơ quan chức năng đã tiến hành triệt phá một đường dây ma túy “khủng”. Cụ thể các lực lượng của Cục trưởng Cục Cảnh sát điều tra tội phạm về ma túy-Bộ Công an (C04), Cục hải quan TPHCM, Tiểu đoàn Cảnh sát đặc nhiệm số 2, Công an TPHCM, Công an tỉnh Bình Dương… chia nhiều mũi, đồng loạt khám xét các kho hàng tại xã Vĩnh Lộc A (huyện Bình Chánh) và một số địa điểm ở các quận Tân Phú, Tân Bình....',
  'Tại các địa điểm nay, nhà chức trách phát hiện khoảng 500 kg ma túy tổng hợp Ketamin trên ôtô 7 chỗ và nhiều tang vật khác. Lực lượng chức năng cũng tiến hành giữ đối tượng Liu Ming Yang (34 tuổi), Jhu Minh Jyun (32 tuổi) và Nguyễn Thị Thu Vân (36 tuổi, cùng quốc tịch Đài Loan); Tô Gia Mỹ (33 tuổi, quốc tịch Trung Quốc).',
  'Theo cơ quan chức năng, đây là đường dây vận chuyển trái phép chất ma túy (ketamine) có số lượng lớn nhất từ trước đến nay về Việt Nam để tuồn ra nước ngoài tiêu thụ. Số ma túy này có giá trị ước tính lên đế

In [42]:
#remove '\n' where applicable and lowercase all of the body-strings
body_postprocessing =  ['' for i in range(len(body_preprocessing))]

for i in range(len(body_preprocessing)):
    for j in body_preprocessing[i]:
        fst_trans = j.lower()
        snd_trans = fst_trans.replace('\n', ' ')
        if j == body_preprocessing[i][0]:
            body_postprocessing[i] += snd_trans
        else:
            body_postprocessing[i] += ' ' + snd_trans
body_postprocessing[:2]

['ma túy trên ôtô. ảnh: hải quan tp hcm. vào ngày 11/5, cơ quan chức năng đã tiến hành triệt phá một đường dây ma túy “khủng”. cụ thể các lực lượng của cục trưởng cục cảnh sát điều tra tội phạm về ma túy-bộ công an (c04), cục hải quan tphcm, tiểu đoàn cảnh sát đặc nhiệm số 2, công an tphcm, công an tỉnh bình dương… chia nhiều mũi, đồng loạt khám xét các kho hàng tại xã vĩnh lộc a (huyện bình chánh) và một số địa điểm ở các quận tân phú, tân bình.... tại các địa điểm nay, nhà chức trách phát hiện khoảng 500 kg ma túy tổng hợp ketamin trên ôtô 7 chỗ và nhiều tang vật khác. lực lượng chức năng cũng tiến hành giữ đối tượng liu ming yang (34 tuổi), jhu minh jyun (32 tuổi) và nguyễn thị thu vân (36 tuổi, cùng quốc tịch đài loan); tô gia mỹ (33 tuổi, quốc tịch trung quốc). theo cơ quan chức năng, đây là đường dây vận chuyển trái phép chất ma túy (ketamine) có số lượng lớn nhất từ trước đến nay về việt nam để tuồn ra nước ngoài tiêu thụ. số ma túy này có giá trị ước tính lên đến gần 500 tỷ đồn

In [74]:
body_preprocessing[3][1]

'Có mặt tại một buổi giao lưu tại TP.HCM với chủ đề “Không trẻ em nào bị bỏ lại phía sau”, TS. Bùi Trân Phượng phân tích, nếu cha mẹ để lỡ các dấu mốc trưởng thành quan trọng của con trẻ sẽ rất khó khăn để kết nối trở lại với các con; nếu trong suốt tuổi thơ đứa trẻ không làm cho phụ huynh đau đầu lần nào thì đó là một đứa trẻ không bình thường. Có thể tuổi vị thành niên là rất khó chịu cả với các bậc phụ huynh lẫn với trẻ, đây là khoảng thời gian quan trọng phát lộ dần con người thực của mỗi cá thể, nên bản thân cá thể đó cũng phải đối diện với những vấn đề mà mình không hiểu; mình không hiểu mình thì làm sao người khác hiểu mình?'

In [75]:
data.body[1]

['Cách đây hơn một năm, tôi từng kêu gọi Mark Zuckerberg, Chủ tịch Hội đồng Quản trị kiêm CEO của Facebook, nên từ chức hoặc bị sa thải.',
 'Dù Facebook đã hứng chịu rất nhiều chỉ trích, nhiều người vào thời điểm đó vẫn cho rằng đòi hỏi Zuckerberg từ chức là khắt khe. Ông chủ Facebook vẫn được tin tưởng sẽ có thể đưa ra những giải pháp đúng đắn để giải quyết triệt để mọi vấn đề.',
 'Một năm trôi qua với biết bao thay đổi. Giờ đây mọi chuyện trở nên sáng tỏ hơn với nhiều người trên toàn thế giới.',
 'Thất bại lớn nhất của Facebook',
 'Sở hữu hơn 58% cổ phiếu có quyền biểu quyết, Zuckerberg nắm giữ những vị trí trọng yếu nhất và thâu tóm hết quyền lực vào tay. Ông chủ Facebook ở trong vị thế không thể bị khuất phục, có thể phủi tay trước mọi trách nhiệm mặc cho đòi hỏi càng cao từ phía cổ đông, các nhóm hoạt động nhân quyền, nhà chức trách cũng như người dùng Facebook toàn cầu trong việc giải quyết tận gốc những vấn đề mạng xã hội này gây ra đối với toàn xã hội.',
 'Bản thân tôi không nắ

In [51]:
len(body_postprocessing[2])

6808

In [52]:
X_pre_processing = []
y = []
l = 0
for i in data.breadcrumbs:
    if len(body_postprocessing[l]) > 40:
        X_pre_processing.append(body_postprocessing[l])
        y.append(i[0])
    l += 1

In [53]:
len(X_pre_processing)

24614

In [54]:
#group words that go together (groups of 2 or more)
X_post_processing = []
for i in X_pre_processing:
    X_post_processing.append(ViTokenizer.tokenize(i))

'phó thủ_tướng trung quốc lưu hạc trao_đổi với tổng_thống donald_trump tại phòng bầu_dục ở nhà trắng hôm 4 - 4 . ông lưu hạc sẽ đến washington ngày 9 - 5 tới để tiếp_tục đàm_phán thương_mại với mỹ . ảnh : reuters hôm 7 - 5 , bộ thương_mại trung quốc thông_báo phó thủ_tướng lưu hạc sẽ dẫn_đầu đoàn đàm_phán trung quốc đến washington vào ngày 9 - 5 để tiếp_tục đàm_phán thương_mại với mỹ . chuyến thăm sẽ diễn ra trong 2 ngày , ngắn hơn so với kế_hoạch trước đó . chuyến thăm diễn ra giữa lúc những bất_ổn treo lơ lững trên bàn đàm_phán mỹ - trung sau khi tổng_thống mỹ donald trump hôm 5 - 5 thông_báo sẽ tăng thuế từ mức 10 % lên 25 % đối_với 200 tỉ đô_la hàng_hóa trung quốc bán sang mỹ mỗi năm vào ngày 10 - 5 để phản_ứng trước việc trung quốc rút lại hầu_hết cam_kết quan_trọng trong dự_thảo thỏa_thuận thương_mại của 2 nước trước đó . truyền_thông nhà_nước trung quốc nói rằng bắc kinh sẽ không nhượng_bộ trước lời đe_dọa tăng thuế của ông trump . trong một bài bình_luận đăng trên tài_khoản mạn

In [96]:
X_post_processing[500]

'nhiều du_khách tại hội an ( quảng_nam ) bày_tỏ sự bất_ngờ khi phát_hiện xung_quanh mặt_trời có vòng ánh_sáng rực_rỡ . được biết , hiện_tượng hiếm thấy này kéo_dài nhiều tiếng đồng_hồ và kết_thúc lúc 13h30 . thời_tiết ở hội an khoảng 31 độ c , trời trong xanh và ít mây nên hiện_tượng này khá dễ quan_sát . nhiều du_khách đã tranh_thủ chụp ảnh , quay lại khoảnh_khắc đáng nhớ . hào_quang mặt_trời xuất_hiện ở hội an khiến du_khách bất_ngờ . ảnh : hương thảo . hương thảo ( hà_nội ) đã không giấu nổi sự bất_ngờ khi nhìn thấy hào_quang : " mình cảm_thấy rất kỳ_diệu vì chưa bao_giờ được nhìn thấy cảnh_tượng này . người dân và du_khách ở đây cũng cảm_thấy rất lạ . họ còn chụp ảnh để ghi lại khoảnh_khắc ấy ngay lập_tức " . trên mạng xã_hội , nhiều du_khách nước_ngoài cũng tỏ ra bất_ngờ khi chia_sẻ hình_ảnh về hiện_tượng này tại hội an . du_khách nước_ngoài ghi lại cảnh_tượng hiếm thấy ở hội an và chia_sẻ lên mạng xã_hội . ảnh chụp màn_hình . một du_khách nước_ngoài khác thể_hiện sự thích_thú trê

In [97]:
y[500]

'Khoa học'

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X_post_processing, y, shuffle = True, test_size = 0.3, random_state = 42)

In [56]:
len(X_post_processing)

24614

In [57]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)
X_wordemb_train = vectorizer.transform(X_train)
X_wordemb_test = vectorizer.transform(X_test)

In [59]:
X_wordemb_train

<17229x70425 sparse matrix of type '<class 'numpy.float64'>'
	with 3607902 stored elements in Compressed Sparse Row format>

In [60]:
X_wordemb_test

<7385x70425 sparse matrix of type '<class 'numpy.float64'>'
	with 1529065 stored elements in Compressed Sparse Row format>

In [62]:
clf_linear = LogisticRegression(random_state=42).fit(X_wordemb_train, y_train)

/Users/letuxuanhao/opt/miniconda3/envs/envi/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [63]:
clf_linear.score(X_wordemb_test, y_test)

0.9260663507109005

In [108]:
# import joblib
from joblib import dump

# dump the pipeline model
dump(clf_linear, filename="text_classification.joblib")

['text_classification.joblib']

In [65]:
clf_linear.predict("Địa phương đang có dịch và thực hiện Chỉ thị số 15, 16 trước mắt, tổ chức dạy học trực tuyến; ưu tiên nguồn lực hỗ trợ học sinh khó khăn không có phương tiện học tập trực tuyến; đổi mới phương thức dạy học giúp trẻ em, học sinh vừa học vừa chơi, hứng thú học tập, chấp hành giãn cách, giảm căng thẳng trong thời gian giãn cách xã hội.")

ValueError: Expected 2D array, got scalar array instead:
array=Địa phương đang có dịch và thực hiện Chỉ thị số 15, 16 trước mắt, tổ chức dạy học trực tuyến; ưu tiên nguồn lực hỗ trợ học sinh khó khăn không có phương tiện học tập trực tuyến; đổi mới phương thức dạy học giúp trẻ em, học sinh vừa học vừa chơi, hứng thú học tập, chấp hành giãn cách, giảm căng thẳng trong thời gian giãn cách xã hội..
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [103]:
text = "Trong ngày, Đà Nẵng có 47 ca mắc COVID-19, 5 ca mắc tại cộng đồng là các trường hợp có triệu chứng đến khám, xét nghiệm tại Trung tâm y tế quận Liên Chiểu và Bệnh viện Hoàn Mỹ Đà Nẵng (lấy mẫu đại diện hộ gia đình trước đó âm tính vào các ngày 24 và 30/8). Liên quan đến chuỗi lây nhiễm từ xét nghiệm hộ gia đình vẫn còn nguy cơ cao với 19 ca mắc mới được ghi nhận trong ngày, gồm 13 F1 đã cách ly và 6 trường hợp trong khu phong tỏa. Ban Chỉ đạo cũng cho biết trong ngày, Bệnh viện dã chiến tại ký túc xá phía tây Thành phố, Trung tâm y tế huyện Hòa Vang và Bệnh viện Phổi Đà Nẵng công bố khỏi bệnh và cho xuất viện 170 bệnh nhân mắc COVID-19. Đáng chú ý, tại Trung tâm y tế huyện Hoà Vang có 48 bệnh nhân xuất viện là con số lớn nhất từ trước đến nay, trong đó có đến 25 bệnh nhân là trẻ em, 6 cháu bé dưới 2 tuổi được chữa khỏi bệnh. Bí thư Thành ủy Đà Nẵng Nguyễn Văn Quảng cho rằng, đây là lần đầu tiên Thành phố thực hiện biện pháp giãn dân để phòng, chống dịch. Việc triển khai đến nay, các biện pháp triển khai cơ bản đáp ứng yêu cầu đề ra, thực hiện chặt chẽ. Sau khi hoàn thành di dời người dân tại các kiệt, hẻm có nhiều ca mắc COVID-19, lực lượng y tế sẽ khẩn trương phun khử khuẩn các kiệt, hẻm, khu vực nóng về COVID-19. Lực lượng công an tăng cường tuần tra, tránh để mất mát tài sản của người dân; giám sát việc chấp hành biện pháp phòng, chống dịch trong thời gian thực hiện giãn dân. Ông Hồ Thuyên, Chủ tịch UBND quận Thanh Khê cho biết, trong hai ngày 3/9 và 4/9, chính quyền phối hợp với lực lượng công an, y tế đã vận động, di dời khẩn cấp gần 500 người dân tại loạt điểm nóng lây lan dịch COVID-19 từ Kiệt K112 đến K292; Kiệt K158 đến K160 giao K112/37 Trần Cao Vân có số ca lây nhiễm cao (thuộc tổ 30, 31) và Kiệt K236, K258 và K264 thuộc 3 tổ (14, 15, 17) đều thuộc phường Tam Thuận, quận Thanh Khê. Đến nay trên địa bàn phường Tam Thuận có trên 230 F0. Người dân các điểm kiệt này được đưa về khu tập trung tại 2 trường tiểu học Bế Văn Đàn và Huỳnh Ngọc Huệ được quận Thanh Khê trưng dụng, bố trí đầy đủ điều kiện cần thiết, bố trí nơi ăn ở để người dân tạm thời ra khỏi khu dân cư chật hẹp, phòng tránh lây lan, lây nhiễm chéo dịch bệnh COVID-19. Đồng thời, để hỗ trợ người dân trong thời gian cách ly, UBND TP. Đà Nẵng cũng vừa có Quyết định về việc phê duyệt hỗ trợ các hộ dân và đối tượng bảo trợ xã hội tập trung trong thời gian Thành phố tiếp tục thực hiện các biện pháp cấp bách phòng, chống dịch, tổng kinh phí hơn 113 tỷ đồng.Các hộ dân trên địa bàn Đà Nẵng (kể cả tạm trú, sinh viên, lao động thuê trọ) chưa nhận được hỗ trợ trong gói hỗ trợ hơn 71 tỷ đồng trước đó, sẽ được hỗ trợ tiền mặt mức 500.000 đồng/hộ. Các đối tượng bảo trợ xã hội nuôi dưỡng tập trung, chi hỗ trợ bằng tiền mặt mức 250.000 đồng/người. Tổng cộng sẽ có 226.225 hộ dân được nhận hỗ trợ lần này. Trước đó, Đà Nẵng đã triển khai gói hỗ trợ hơn 71 tỷ đồng cho hơn 142.000 hộ dân."

In [109]:
def transform_text(text):
    string = ViTokenizer.tokenize(text.lower().replace('\n', ' '))
    string_word_emb = vectorizer.transform([string])
    return string_word_emb

In [110]:
clf_linear.predict(transform_text(text))

array(['Pháp luật'], dtype='<U10')